In [4]:
# read sqlite3
import sqlite3
import anki
from anki.collection import Collection
import re
import os
import zipfile
import termcolor
import tqdm
import csv
import shutil
# import pandas as pd

MEDIA_FOLDER = '/Users/AlexG/Library/Application Support/Anki2/User 1/collection.media'
PKG_PATH = '/Users/AlexG/Downloads/MyPaperNotes.apkg'
PKG_PATH = '/Users/AlexG/Downloads/ivl10.apkg'
PKG_PATH = '/Users/AlexG/Downloads/极品GRE红宝书.apkg'
outdir = PKG_PATH + '-decompress'
# outdir = '/Users/AlexG/Downloads/ivl10-decompress'
# ANKI2_PATH = '/Users/AlexG/Downloads/ivl10-decompress/collection.anki2'
ANKI21_PATH = outdir + '/collection.anki21'
# EXPORT_PATH_WEBSITE = '/Users/AlexG/Documents/GitHub/typingpractices/website/onepager/onepager/templates/onepager/cards.html'
EXPORT_PATH_WEBSITE = '/Users/AlexG/Documents/GitHub/anki_html/docs/gre_cards.html'

def unzip_anki(pkg_file_path, outdir):
    # Create the extract directory if it doesn't exist
    os.makedirs(outdir, exist_ok=True)

    # Unzip the .apkg file
    with zipfile.ZipFile(PKG_PATH, 'r') as zip_ref:
        zip_ref.extractall(outdir)

    print(f"Extracted files to {outdir}")

class ImageSrcUrls(object):
    def __init__(self):
        self.purge_script_path = '/Users/AlexG/Documents/GitHub/anki_html/run_images.sh'
        self.git_push_script_path = '/Users/AlexG/Documents/GitHub/anki_html/run_upload.sh'
        self.source_image_folder = MEDIA_FOLDER
        self.dest_image_folder = '/Users/AlexG/Documents/GitHub/anki_html/docs/images'
        self.IMG_PAT = re.compile(r'<img\b.*\bsrc="')
        self.SRC_PAT = re.compile(r'<img\b.*\bsrc="([^"]+)"')
        self.srcs = []
        self.exts = set()

    def replace_image_url(self, text):
        res = self.SRC_PAT.search(text)
        if res:
            for s in res.groups():
                self.srcs.append(s)
                ext = os.path.splitext(s)[1]
                self.exts.add(ext)
                print(termcolor.colored(s, 'red'))

        return self.IMG_PAT.sub('<img src="./images/', text)
    
    def purge_images(self):
        os.system(f"bash {self.purge_script_path}") 

    def git_push(self):
        os.system(f"bash {self.git_push_script_path}")
        
    def move_images(self):
        self.purge_images()
        for src in tqdm.tqdm(self.srcs):
            shutil.copyfile(
                os.path.join(self.source_image_folder, src), 
                os.path.join(self.dest_image_folder, src)
            )


def format_cards_to_html(cards, collection, outpath, max_text_length=80, max_cards=200):
    data = []
    image_src = ImageSrcUrls()
    for cid in tqdm.tqdm(cards):
        card = collection.get_card(cid)
        note = card.note()
        fields = note.fields
        # fields[1] = fields[1][:20]
        # fields = [field if len(field) <= max_text_length else field[:max_text_length] + '...' for field in fields]
        data.append((card.ivl, fields))

    data.sort()
    data = [[image_src.replace_image_url(field) for field in fields] 
            for _, fields in data[:max_cards]]
    print(image_src.exts)
    image_src.move_images()
    # with open(outpath+'.csv', 'w') as f:
    #     wt = csv.writer(f)
    #     wt.writerows([v[:3] for v in data])
    rows = ['<tr><td class="indextd">{}</td><td>{}</td></tr>\n'.format(fi, '</td><td class="fixwtd">'.join(field[:3])) for fi, field in enumerate(data)]
    html = f'''
    <head>
        <style>
            table {{
                --color: #d0d0f5;
                margin: 3em;
            }}

            thead,
            tfoot {{
                background: var(--color);
            }}

            tbody tr:nth-child(even) {{
                background: color-mix(in srgb, var(--color), transparent 60%);
            }}
            div, p, td {{
                width: 200px;
                word-wrap: break-word;
                white-space: normal;
            }}
            .fixwtd {{
                width: 200px;
                word-wrap: break-word;
                white-space: normal;
            }}
            .indextd {{
                width: 10px;
                word-wrap: break-word;
                white-space: normal;
            }}
        </style>
        <link rel="stylesheet" href="css/fancy-button.css">
    </head>
    <body>
        <button onclick="changeWidth(0.5)" class="button-74">Half Column Width</button>
        <button onclick="hideRows(20)" class="button-73 fix-width-button">Hide Rows</button>
        <table id="cards">
          {"".join(rows)}
        </table>
        <script src="js/change-width.js"></script>
        <script src="js/hide-rows.js"></script>
    </body>
    '''
    with open(outpath, 'w') as f:
        f.write(html)

    # git push
    image_src.git_push()

unzip_anki(PKG_PATH, outdir)
col = Collection(ANKI21_PATH)
# Fetch all decks in the collection
decks = col.decks.all()
print("Decks in collection:")
for deck in decks:
    print(f"Deck ID: {deck['id']}, Name: {deck['name']}")

try:
    print(dir(col))
    print('total card count: ', col.card_count())
    # print('note count: ', col.node_count())

    query = 'prop:ivl<=3'
    query = ''
    cards = col.find_cards(query)
    print(termcolor.colored(f'query {query} find {len(cards)} cards', 'green'))

    outpath = os.path.join(outdir, 'cards.html')
    # format_cards_to_html(cards, col, outpath)
    format_cards_to_html(cards, col, EXPORT_PATH_WEBSITE)

    # Fetch all notes (cards content) in the collection
    # print("\nNotes in collection:")
    # for note_id in col.db.list("select id from notes"):
    #     note = col.get_note(note_id)
    #     fields = note.fields
    #     print(f"Note ID: {note_id}, Fields: {fields}")

    # # Fetch all cards (metadata) in the collection
    # print("\nCards in collection:")
    # for card_id in col.db.list("select id from cards"):
    #     card = col.get_card(card_id)
    #     print(f"Card ID: {card.id}, Note ID: {card.nid}, Deck ID: {card.did}, Type: {card.type}")

    
except Exception as e:
    print(f"Error: {e}")
finally:
    # Close the collection
    col.close()

col.close()

Extracted files to /Users/AlexG/Downloads/极品GRE红宝书.apkg-decompress
Decks in collection:
Deck ID: 1685695741424, Name: 极品GRE红宝书
Deck ID: 1, Name: Default
['__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_backend', '_build_sort_mode', '_check_backend_undo_status', '_clear_caches', '_deprecated_aliases', '_deprecated_attributes', '_load_scheduler', '_logRem', '_pb_search_separator', '_remNotes', '_startReps', '_supported_scheduler_versions', 'abort_media_sync', 'abort_sync', 'addNote', 'add_custom_undo_entry', 'add_image_occlusion_note', 'add_image_occlusion_notetype', 'add_note', 'add_notes', 'after_note_updates', 'all_browser_columns', 'all_config', 'autosave', 

100%|██████████| 6856/6856 [00:00<00:00, 6922.97it/s]
ls: images: No such file or directory


paste-dea8bb55b40d6a755fc9c8190c7bcfa0cff0cbd4.jpg
paste-c0dd42e84aaaf23e322d5e9b978e109d75b170c6.jpg
images-e232c45b1db6f1af89fc1688113275e37baaa329.jpg
paste-3734f2bb9ca20bb7bd2e2c08e753015ab130772f.jpg
images-1c3d9d87dd6dd61d31504b0212b30cbdab24c981.jpg
{'.jpg'}


100%|██████████| 5/5 [00:00<00:00, 110.47it/s]


20240921(01:05:28)
[main ae7c857] update cards.html on date 20240921(01:05:28)
 4 files changed, 202 insertions(+), 111 deletions(-)
 create mode 100644 docs/images/images-1c3d9d87dd6dd61d31504b0212b30cbdab24c981.jpg
 delete mode 100644 docs/images/paste-ebebd3acddf1167667e5afbea68bd1142c6c8447.jpg


To github.com:pengfeigao2021/anki_html.git
 + 5d339c1...ae7c857 main -> main (forced update)
